In [29]:
from Data_Handler.DataReader import DataReader
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from impressions import Impressions
from hybrid import *
from tqdm import tqdm
from evaluator import evaluate
import pandas as pd
import numpy as np
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.DataIO import DataIO
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from skopt.space import Real, Integer, Categorical
import os

In [30]:
dataReader = DataReader()

target = dataReader.load_target()

URM = dataReader.load_augmented_binary_urm()
URM_aug,icm = dataReader.pad_with_zeros_ICMandURM(URM)

URM_train_aug, URM_validation = split_train_in_two_percentage_global_sample(URM_aug, train_percentage = 0.9)
URM_train_pow = dataReader.stackMatrixes(URM_train_aug)
UCM = dataReader.load_aug_ucm()


evaluator_validation = EvaluatorHoldout(URM_validation, [10])

>>> number of target users: 41116
Making augmented URM and ICM of the same shape...
EvaluatorHoldout: Ignoring 2857 ( 6.9%) Users that have less than 1 test interactions


In [31]:
UserKNNCF = UserKNNCFRecommender(URM_train_aug)
UserKNNCF.fit()

UserKNNCFRecommender: URM Detected 3461 (12.4%) items with no interactions.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 41629 (100.0%), 569.60 column/sec. Elapsed time 1.22 min


In [32]:
ItemKNNCF = ItemKNNCFRecommender(URM_train_pow)
ItemKNNCF.fit()

Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 27968 (100.0%), 567.72 column/sec. Elapsed time 49.26 sec


In [33]:
RP3beta_aug = RP3betaRecommender(URM_train_aug)
RP3beta_aug.fit(alpha=0.6951524535062256,beta=0.39985511876562174, topK=82, normalize_similarity=True)

RP3betaRecommender: URM Detected 3461 (12.4%) items with no interactions.
RP3betaRecommender: Similarity column 27968 (100.0%), 3570.52 column/sec. Elapsed time 7.83 sec


In [34]:
RP3beta_pow = RP3betaRecommender(URM_train_pow)
RP3beta_pow.fit(alpha=0.3648761546066018,beta=0.5058870363874656, topK=480, normalize_similarity=True)

RP3betaRecommender: Similarity column 27968 (100.0%), 1349.05 column/sec. Elapsed time 20.73 sec


In [35]:
S_SLIM = SLIMElasticNetRecommender(URM_train_pow)
S_SLIM.fit()

SLIMElasticNetRecommender: Processed 5685 (20.3%) in 5.00 min. Items per second: 18.95
SLIMElasticNetRecommender: Processed 11709 (41.9%) in 10.00 min. Items per second: 19.51
SLIMElasticNetRecommender: Processed 17742 (63.4%) in 15.00 min. Items per second: 19.71
SLIMElasticNetRecommender: Processed 23773 (85.0%) in 20.00 min. Items per second: 19.81
SLIMElasticNetRecommender: Processed 27968 (100.0%) in 21.09 min. Items per second: 22.10


In [36]:
EASE_R = EASE_R_Recommender(URM_train_aug)
EASE_R.fit()

EASE_R_Recommender: URM Detected 3461 (12.4%) items with no interactions.
EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 6.21 min


In [37]:
Hybrid_SSLIM_EASER = Hybrid_SSLIM_EASER(URM_train_aug, URM_train_pow, S_SLIM, EASE_R)
Hybrid_SSLIM_EASER.fit(SSLIM_weight=0.563368095251961, EASE_R_weight=0.0)

Hybrid_layer1: URM Detected 3461 (12.4%) items with no interactions.


In [38]:
Hybrid_SSLIM_RP3B_aug = Hybrid_SSLIM_RP3B_aug(URM_train_aug, URM_train_pow, S_SLIM, RP3beta_aug)
Hybrid_SSLIM_RP3B_aug.fit(SSLIM_weight= 0.8157521052599057, RP3B_weight=0.22946157569349823)

Hybrid_SSLIM_RP3B_aug: URM Detected 3461 (12.4%) items with no interactions.


In [39]:
HybridRecommender_5 = HybridRecommender_5(URM_train_aug, URM_train_pow, UserKNNCF, RP3beta_pow, S_SLIM, EASE_R)
HybridRecommender_5.fit()

Hybrid_Recommender_5: URM Detected 3461 (12.4%) items with no interactions.


In [40]:
recommender = Hybrid_of_Hybrids(URM_train_aug, URM_train_pow, UCM, HybridRecommender_5, Hybrid_SSLIM_EASER, Hybrid_SSLIM_RP3B_aug)
recommender.fit(Hybrid_1_tier1_weight=0.09906699744028494, Hybrid_1_tier2_weight=0.2713344803469841, Hybrid_2_tier2_weight = 0.6798129679473879)

Hybrid_of_Hybrids: URM Detected 3461 (12.4%) items with no interactions.


In [43]:
f = open("submission.csv", "w+")
f.write("user_id,item_list\n")
recommended_items_for_each_user = {}
for user_id in tqdm(target):
    recommended_items = recommender.recommend(
        user_id, cutoff=10, remove_seen_flag=True)
    # recommended_items=impressions.update_ranking(user_id,recommended_items,dataReader)
    recommended_items_for_each_user[int(user_id)] = recommended_items
    well_formatted = " ".join([str(x) for x in recommended_items])
    f.write(f"{user_id}, {well_formatted}\n")

# Evaluate recommended items
map = evaluate(recommended_items_for_each_user, URM_validation, target)
print('MAP score: {}'.format(map))

  0%|          | 1/41116 [00:00<05:31, 123.97it/s]


AttributeError: 'Hybrid_of_Hybrids' object has no attribute 'Hybrid_2_tier1_weight'